# Scala Experiment

In [1]:
%%init_spark
launcher.master = "spark://spark-master:7077"
launcher.num_executors = 2
launcher.executor_cores = 2
launcher.executor_memory = '4G'
launcher.packages = ["com.amazon.deequ:deequ:1.2.2-spark-3.0"]

In [2]:
spark

Intitializing Scala interpreter ...

Spark Web UI available at http://7847ea2e55be:4040
SparkContext available as 'sc' (version = 3.1.1, master = spark://spark-master:7077, app id = app-20210522114526-0001)
SparkSession available as 'spark'


res0: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@1a869eca


In [3]:
spark.version

res1: String = 3.1.1


In [4]:
val path = "/opt/spark-data"

path: String = /opt/spark-data


In [13]:
import org.apache.spark.sql.SparkSession
import com.amazon.deequ.VerificationSuite
import com.amazon.deequ.checks.{Check, CheckLevel, CheckStatus}
import com.amazon.deequ.constraints.ConstraintStatus


import org.apache.spark.sql.SparkSession
import com.amazon.deequ.VerificationSuite
import com.amazon.deequ.checks.{Check, CheckLevel, CheckStatus}
import com.amazon.deequ.constraints.ConstraintStatus


In [7]:
// Open the parquet and have a look
val s3_stats = spark.read.parquet(path + "/s3logs")

s3_stats: org.apache.spark.sql.DataFrame = [bucketowner: string, bucket: string ... 27 more fields]


In [9]:
s3_stats.printSchema()

root
 |-- bucketowner: string (nullable = true)
 |-- bucket: string (nullable = true)
 |-- requestdatetime: string (nullable = true)
 |-- remoteip: string (nullable = true)
 |-- requester: string (nullable = true)
 |-- requestid: string (nullable = true)
 |-- operation: string (nullable = true)
 |-- key: string (nullable = true)
 |-- request: string (nullable = true)
 |-- httpstatus: string (nullable = true)
 |-- errorcode: string (nullable = true)
 |-- bytessent: integer (nullable = true)
 |-- objectsize: integer (nullable = true)
 |-- totaltime: integer (nullable = true)
 |-- turnaroundtime: integer (nullable = true)
 |-- referrer: string (nullable = true)
 |-- useragent: string (nullable = true)
 |-- versionid: string (nullable = true)
 |-- hostid: string (nullable = true)
 |-- sigv: string (nullable = true)
 |-- ciphersuite: string (nullable = true)
 |-- authtype: string (nullable = true)
 |-- endpoint: string (nullable = true)
 |-- tlsversion: string (nullable = true)
 |-- request

In [17]:
// try deequ test
val verificationResult = VerificationSuite()
    .onData(s3_stats)
    .addCheck(
        Check(CheckLevel.Error, "unit testing my data")
            .isComplete("requestdatetime")
            .isComplete("key")
        )
    .run()

verificationResult: com.amazon.deequ.VerificationResult = VerificationResult(Success,Map(Check(Error,unit testing my data,List(CompletenessConstraint(Completeness(requestdatetime,None)), CompletenessConstraint(Completeness(key,None)))) -> CheckResult(Check(Error,unit testing my data,List(CompletenessConstraint(Completeness(requestdatetime,None)), CompletenessConstraint(Completeness(key,None)))),Success,List(ConstraintResult(CompletenessConstraint(Completeness(requestdatetime,None)),Success,None,Some(DoubleMetric(Column,Completeness,requestdatetime,Success(1.0)))), ConstraintResult(CompletenessConstraint(Completeness(key,None)),Success,None,Some(DoubleMetric(Column,Completeness,key,Success(1.0))))))),Map(Completeness(requestdatetime,None) -> DoubleMetric(Column,Completeness,requestdateti...


In [18]:
if (verificationResult.status == CheckStatus.Success) {
  println("The data passed the test, everything is fine!")
} else {
  println("We found errors in the data:\n")

  val resultsForAllConstraints = verificationResult.checkResults
    .flatMap { case (_, checkResult) => checkResult.constraintResults }

  resultsForAllConstraints
    .filter { _.status != ConstraintStatus.Success }
    .foreach { result => println(s"${result.constraint}: ${result.message.get}") }
}

The data passed the test, everything is fine!


In [ ]:
// stop the session
spark.stop()